# Basis Test

In [1]:
# Import package
using LinearAlgebra, Plots, Test
DEV = true
if DEV    # use local package
    include("../src/EDKit.jl")
    using .EDKit
else      # use EDKit in the Pkg system
    using EDKit
end

## ProjectedBasis

In [2]:
@testset "XY Model" begin
    L = 10
    mat = spin((1, "xx"), (1, "yy"))
    E = zeros(2^L)
    P = 0
    for n = 0:L
        basis = ProjectedBasis(L=L, N=n, small_N=false)
        if (l = size(basis, 1)) > 0
            vals = trans_inv_operator(mat, 2, basis) |> Hermitian |> eigvals
            E[P+1:P+l] = vals
            P += l
        end
    end
    @test P == 2^L
    vals = trans_inv_operator(mat, 2, L) |> Hermitian |> eigvals
    @test vals ≈ sort!(E)
end;

@testset "XY: small N" begin
    L = 10
    mat = spin((1, "xx"), (1, "yy"))
    E = zeros(2^L)
    P = 0
    for n = 0:L
        basis = ProjectedBasis(L=L, N=n, small_N=true)
        if (l = size(basis, 1)) > 0
            vals = trans_inv_operator(mat, 2, basis) |> Hermitian |> eigvals
            E[P+1:P+l] = vals
            P += l
        end
    end
    @test P == 2^L
    vals = trans_inv_operator(mat, 2, L) |> Array |> Hermitian |> eigvals
    @test vals ≈ sort!(E)
end;

Test Summary: | Pass  Total  Time
XY Model      |    2      2  1.3s
Test Summary: | 

Pass  Total  Time
XY: small N   |    2      2  0.1s


Bench mark for small-N method:

In [3]:
@time ProjectedBasis(L=28, N=4, small_N=false);
@time ProjectedBasis(L=28, N=4, small_N=true);
@time ProjectedBasis(L=12, N=4, base=5, small_N=false);
@time ProjectedBasis(L=12, N=4, base=5, small_N=true);

  2.278783 seconds (115 allocations: 624.836 KiB)
  0.022741 seconds (727.96 k allocations: 23.238 MiB)


  0.774164 seconds (132 allocations: 85.398 KiB)
  0.000906 seconds (22.67 k allocations: 731.281 KiB)


In [4]:
@time ProjectedBasis(L=16, N=4, base=5, small_N=true);
@time ProjectedBasis(L=20, N=4, base=5, small_N=true);
@time ProjectedBasis(L=30, N=4, base=5, small_N=true);

  0.002916 seconds (89.16 k allocations: 2.529 MiB)
  0.008637 seconds (260.35 k allocations: 6.935 MiB)


  0.055742 seconds (1.86 M allocations: 45.831 MiB, 11.72% gc time)


In [5]:
a = ProjectedBasis(BigInt, L=30, N=4, base=5, small_N=true)
a

ProjectedBasis{BigInt}([0, 0, 0, 0, 0, 0, 0, 0, 0, 0  …  0, 0, 0, 0, 0, 0, 0, 0, 0, 0], BigInt[5, 9, 13, 17, 21, 29, 33, 37, 41, 53  …  558793640136718750001, 558794021606445312501, 558795928955078125001, 558805465698242187501, 558853149414062500001, 559091567993164062501, 560283660888671875001, 566244125366210937501, 596046447753906250001, 745058059692382812501], 5)

## TranslationalBasis

In [6]:
@testset "XY Model" begin
    L = 10
    θ = 0.34
    expθ = exp(-1im*θ)
    mat = spin((expθ, "+-"), (1/expθ, "-+"), (1, "z1"), (1, "1z"))
    E = zeros(2^L)
    P = 0
    for n = 0:L, k = 0:L-1
        basis = TranslationalBasis(L=L, N=n, k=k, small_N=false)
        if (l = size(basis,1)) > 0
            vals = trans_inv_operator(mat, 2, basis) |> Array |> Hermitian |> eigvals
            E[P+1:P+l] = vals
            P += l
        end
    end
    @test P == 2^L
    vals = trans_inv_operator(mat, 2, L) |> Array |> Hermitian |> eigvals
    @test vals ≈ sort!(E)
end
@testset "XY: small-N" begin
    L = 10
    θ = 0.34
    expθ = exp(-1im*θ)
    mat = spin((expθ, "+-"), (1/expθ, "-+"), (1, "z1"), (1, "1z"))
    E = zeros(2^L)
    P = 0
    for n = 0:L, k = 0:L-1
        basis = TranslationalBasis(L=L, N=n, k=k)
        if (l = size(basis,1)) > 0
            vals = trans_inv_operator(mat, 2, basis) |> Array |> Hermitian |> eigvals
            E[P+1:P+l] = vals
            P += l
        end
    end
    @test P == 2^L
    vals = trans_inv_operator(mat, 2, L) |> Array |> Hermitian |> eigvals
    @test vals ≈ sort!(E)
end;

Test Summary: | Pass  Total  Time
XY Model      |    2      2  0.9s
Test Summary: | 

Pass  Total  Time
XY: small-N   |    2      2  0.4s


Benchmark

In [7]:
@time TranslationalBasis(L=28, N=4, k=0, small_N=false);
@time TranslationalBasis(L=28, N=4, k=0, small_N=true);

  3.755735 seconds (109 allocations: 147.562 KiB)
  0.026756 seconds (768.91 k allocations: 24.226 MiB)


In [8]:
@time b=TranslationalBasis(BigInt, L=40, N=4, k=0, small_N=true);
b.I

  3.581278 seconds (75.78 M allocations: 1.238 GiB, 20.40% gc time, 3.49% compilation time)


2290-element Vector{BigInt}:
         16
         24
         28
         30
         40
         44
         46
         52
         54
         58
          ⋮
  537133314
  537133570
  537134082
  537395458
  537395714
  537396226
  537920002
  537920514
 1074791426

## TranslationParityBasis

In [9]:
@testset "XY Model" begin
    mat = spin((1, "+-"), (1, "-+"), (1, "z1"), (1, "1z"))
    for L = 2:2:10
        for n = 0:L, k in [0, L ÷ 2]
            be = TranslationParityBasis(N=n, k=k, p=+1, L=L, small_N=false)
            bo = TranslationParityBasis(N=n, k=k, p=-1, L=L, small_N=false)
            ba = TranslationalBasis(N=n, k=k, L=L, small_N=false)
            ve = trans_inv_operator(mat, 2, be) |> Array |> Hermitian |> eigvals
            vo = trans_inv_operator(mat, 2, bo) |> Array |> Hermitian |> eigvals
            va = trans_inv_operator(mat, 2, ba) |> Array |> Hermitian |> eigvals
            E = sort(vcat(ve, vo))
            @test norm(E-va) ≈ 0.0 atol = 1e-12
            #be2 = TranslationParityBasis(N=n, k=k, p=+1, L=L, small_N=false)
            #bo2 = TranslationParityBasis(N=n, k=k, p=-1, L=L, small_N=false)
            #@test be2.I == be.I && be2.R == be.R
            #@test bo2.I == bo.I && bo2.R == bo.R
        end
    end
end
@testset "XY: small-N" begin
    mat = spin((1, "+-"), (1, "-+"), (1, "z1"), (1, "1z"))
    for L = 2:2:10
        for n = 0:L, k in [0, L ÷ 2]
            be = TranslationParityBasis(N=n, k=k, p=+1, L=L)
            bo = TranslationParityBasis(N=n, k=k, p=-1, L=L)
            ba = TranslationalBasis(N=n, k=k, L=L)
            ve = trans_inv_operator(mat, 2, be) |> Array |> Hermitian |> eigvals
            vo = trans_inv_operator(mat, 2, bo) |> Array |> Hermitian |> eigvals
            va = trans_inv_operator(mat, 2, ba) |> Array |> Hermitian |> eigvals
            E = sort(vcat(ve, vo))
            @test norm(E-va) ≈ 0.0 atol = 1e-12
        end
    end
end;

Test Summary: | Pass  Total  Time
XY Model      |   70     70  0.3s
Test Summary: | 

Pass  Total  Time
XY: small-N   |   70     70  0.2s


Benchmark

In [10]:
@time b=TranslationParityBasis(BigInt, L=40, N=4, k=0, p=-1, small_N=true);
b.I

  4.019973 seconds (91.39 M allocations: 1.486 GiB, 21.35% gc time, 0.62% compilation time)


1050-element Vector{BigInt}:
        24
        40
        44
        72
        76
        78
        84
       136
       140
       142
         ⋮
 268566786
 268567042
 268567554
 268697730
 268697858
 268698114
 268960002
 537133314
 537395714